# Concept 2: Evaluating AI Agents - Starter Notebook

**Objective**: Learn to evaluate AI agents using industry-standard agentic RAG metrics.

**Prerequisites**: Complete Concept 1 (Finance Memory Agent)

## 🎯 What You'll Learn

This notebook focuses on **evaluation methodology** rather than agent building. You'll implement:

**Top 3 Agentic RAG Metrics:**
- 🎯 **Factual Accuracy** (40% weight): LLM-based correctness scoring
- 📝 **Citation/Source Compliance** (30% weight): Source attribution quality
- 🔍 **Retrieval Relevance** (30% weight): Document retrieval effectiveness with precision/recall

**Evaluation Framework:**
- Load and analyze the golden dataset (50 banking Q&A pairs)
- Integrate with finance memory agent from Concept 1
- Implement production-ready evaluation metrics
- Generate performance insights and recommendations

## 📊 About the Golden Dataset

The dataset was generated using `data/generate_golden_dataset.py` and contains:

**50 Banking Q&A Pairs** across categories:
- Wire transfers, account benefits, deposits, fees, ATM services
- Credit products, savings, lending, business services, security

**Dataset Fields:**
- `question_id`: Unique identifier (e.g., "wire_001")
- `question`: Banking question for the agent
- `correct_answer`: Ground truth response
- `relevant_doc_ids`: Documents that should be retrieved
- `category`: Banking domain for analysis
- `difficulty`: easy/medium/hard for stratified evaluation
- `should_have_citation`: Whether answer requires source attribution
- `expected_retrieval_keywords`: Keywords for retrieval assessment

Let's get started! 🚀

## 🔧 Setup and Imports

Import required libraries and configure the evaluation environment.

In [ ]:
# Import required libraries
import os
import sys
import json
import csv
from typing import List, Dict, Any, Optional, Tuple
import re
from pathlib import Path

# Data handling
import pandas as pd
import numpy as np

# OpenAI for LLM
from openai import OpenAI

# LlamaIndex for RAG (you'll implement this)
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI as LlamaOpenAI
from llama_index.core.retrievers import VectorIndexRetriever

# Environment variables
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

# Configure LlamaIndex settings
Settings.llm = LlamaOpenAI(
    model="gpt-4o-mini", 
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openai.vocareum.com/v1"
)
Settings.embed_model = OpenAIEmbedding(api_key=os.getenv("OPENAI_API_KEY"))

print("🔧 Evaluation Environment Setup:")
print(f"   ✅ OpenAI API Key: {'✓ Configured' if os.getenv('OPENAI_API_KEY') else '❌ Missing'}")
print("   📊 Focus: Top 3 agentic RAG evaluation metrics")
print("   🏦 Domain: Banking policy Q&A evaluation")
print("   📈 Dataset: 50 golden standard labeled examples")

## 📊 Load and Explore Golden Dataset

Load the evaluation dataset generated by `data/generate_golden_dataset.py` and understand its structure.

In [ ]:
def load_golden_dataset() -> Tuple[List[Dict], List[Dict]]:
    """Load the golden Q&A dataset and policy documents"""
    
    # Load Q&A dataset
    qa_data = []
    with open('data/banking_qa_golden_dataset.csv', 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            qa_data.append({
                'question_id': row['question_id'],
                'question': row['question'],
                'correct_answer': row['correct_answer'],
                'relevant_doc_ids': row['relevant_doc_ids'].split('|') if row['relevant_doc_ids'] else [],
                'category': row['category'],
                'difficulty': row['difficulty'],
                'should_have_citation': row['should_have_citation'].lower() == 'true',
                'expected_retrieval_keywords': row['expected_retrieval_keywords'].split('|') if row['expected_retrieval_keywords'] else []
            })
    
    # Load policy documents
    with open('data/banking_policy_documents.json', 'r', encoding='utf-8') as f:
        policy_docs = json.load(f)
    
    return qa_data, policy_docs

# Load the datasets
print("📊 Loading Golden Evaluation Dataset...")
GOLDEN_QA_DATASET, POLICY_DOCUMENTS = load_golden_dataset()

# Explore the dataset structure
qa_df = pd.DataFrame(GOLDEN_QA_DATASET)

print("\n📋 Dataset Overview:")
print(f"   Total questions: {len(GOLDEN_QA_DATASET)}")
print(f"   Policy documents: {len(POLICY_DOCUMENTS)}")
print(f"   Categories: {qa_df['category'].nunique()}")
print(f"   Difficulty levels: {sorted(qa_df['difficulty'].unique())}")

print(f"\n📊 Category Distribution:")
category_counts = qa_df['category'].value_counts()
for category, count in category_counts.items():
    print(f"   {category}: {count} questions")

print(f"\n🎯 Sample Question:")
sample_q = GOLDEN_QA_DATASET[0]
print(f"   ID: {sample_q['question_id']}")
print(f"   Q: {sample_q['question']}")
print(f"   A: {sample_q['correct_answer']}")
print(f"   Category: {sample_q['category']} | Difficulty: {sample_q['difficulty']}")
print(f"   Should cite: {sample_q['should_have_citation']}")
print(f"   Relevant docs: {sample_q['relevant_doc_ids']}")

## 🧠 Finance Memory Agent Integration

**Provided**: Simplified version of the finance memory agent from Concept 1 for evaluation purposes.

*Note: In a real implementation, you'd import the complete agent from Concept 1.*

In [ ]:
# Simplified Finance Memory Agent for Evaluation
# (In practice, this would be imported from Concept 1)

from datetime import datetime
import uuid

class MemoryEntry:
    """Memory entry for banking policy information"""
    def __init__(self, topic: str, fact_text: str, source: str, weight: float = 1.0):
        self.id = str(uuid.uuid4())
        self.topic = topic
        self.fact_text = fact_text
        self.source = source
        self.weight = weight
        self.created_at = datetime.now()

class FinanceMemoryManager:
    """Simplified memory manager from Concept 1"""
    
    def __init__(self):
        self.memories = {}
    
    def add_banking_policies(self, policy_documents: List[Dict]):
        """Add banking policy documents to memory"""
        for doc in policy_documents:
            memory = MemoryEntry(
                topic=doc['category'],
                fact_text=f"{doc['title']}: {doc['content']}",
                source=doc['doc_id'],
                weight=2.0
            )
            self.memories[memory.id] = memory
    
    def retrieve_memories(self, query: str, top_k: int = 2) -> List[MemoryEntry]:
        """Simple keyword-based memory retrieval"""
        query_lower = query.lower()
        scored_memories = []
        
        for memory in self.memories.values():
            score = 0
            fact_lower = memory.fact_text.lower()
            for word in query_lower.split():
                if len(word) > 3 and word in fact_lower:
                    score += memory.weight
            if score > 0:
                scored_memories.append((memory, score))
        
        scored_memories.sort(key=lambda x: x[1], reverse=True)
        return [mem for mem, score in scored_memories[:top_k]]

class EvaluationAgent:
    """Finance assistant with memory + RAG for evaluation"""
    
    def __init__(self, memory_manager: FinanceMemoryManager, rag_retriever):
        self.memory = memory_manager
        self.rag_retriever = rag_retriever
        self.client = client
    
    def answer_question(self, question: str) -> Dict[str, Any]:
        """Answer using both memory and RAG retrieval"""
        
        # Get memory context
        memory_results = self.memory.retrieve_memories(question, top_k=2)
        memory_context = "\n".join([f"Memory: {mem.fact_text}" for mem in memory_results])
        
        # Get RAG context
        rag_results = self.rag_retriever.retrieve(question)
        rag_context = "\n".join([f"Document: {node.text}" for node in rag_results])
        
        # Combine contexts
        combined_context = f"Memory Context:\n{memory_context}\n\nDocument Context:\n{rag_context}"
        
        # Generate answer
        prompt = f"""
You are a banking policy assistant with access to memory and documents.

Context:\n{combined_context}

Question: {question}

Instructions:
- Provide accurate answers based on the context
- Include source references in [brackets] for factual claims
- If information is missing, say "I don't have that information"
- Keep responses professional and concise
"""
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1
            )
            
            return {
                "question": question,
                "answer": response.choices[0].message.content,
                "memory_sources": [mem.source for mem in memory_results],
                "rag_sources": [node.metadata.get('source', 'unknown') for node in rag_results],
                "memory_count": len(memory_results),
                "rag_count": len(rag_results),
                "tokens_used": response.usage.total_tokens,
                "retrieved_nodes": rag_results
            }
        except Exception as e:
            return {
                "question": question,
                "answer": f"Error: {str(e)}",
                "memory_sources": [], "rag_sources": [],
                "memory_count": 0, "rag_count": 0,
                "tokens_used": 0, "retrieved_nodes": []
            }

print("🧠 Finance Memory Agent Components Ready:")
print("   ✅ MemoryEntry and FinanceMemoryManager")
print("   ✅ EvaluationAgent with memory + RAG integration")
print("   🔗 Ready for evaluation framework")

## 🔍  LlamaIndex RAG Setup

**Provided**: LlamaIndex RAG system initialization with banking policy documents.

**Focus**: This setup is provided so you can focus on learning evaluation methodology rather than RAG implementation.

In [ ]:
def setup_llamaindex_rag(policy_documents: List[Dict]) -> VectorIndexRetriever:
    """Set up LlamaIndex RAG system with banking policy documents"""
    
    # Convert policy documents to LlamaIndex Document objects
    documents = []
    for doc in policy_documents:
        # Create document with metadata
        document = Document(
            text=f"{doc['title']}\n\n{doc['content']}",
            metadata={
                'doc_id': doc['doc_id'],
                'title': doc['title'],
                'category': doc['category'],
                'source': doc['doc_id'],
                'keywords': ','.join(doc['relevance_keywords'])
            }
        )
        documents.append(document)
    
    # Create vector index
    print("   🔍 Building LlamaIndex vector store...")
    vector_index = VectorStoreIndex.from_documents(documents)
    
    # Create retriever
    retriever = VectorIndexRetriever(
        index=vector_index,
        similarity_top_k=3  # Retrieve top 3 most relevant documents
    )
    
    print(f"   ✅ LlamaIndex RAG initialized with {len(documents)} documents")
    return retriever

# Set up the RAG system
print("🔍 Setting up LlamaIndex RAG...")
llamaindex_retriever = setup_llamaindex_rag(POLICY_DOCUMENTS)

# Initialize memory manager and agent
print("🧠 Initializing Finance Memory Manager...")
memory_manager = FinanceMemoryManager()
memory_manager.add_banking_policies(POLICY_DOCUMENTS)
evaluation_agent = EvaluationAgent(memory_manager, llamaindex_retriever)

print("\n✅ RAG System Ready!")
print(f"   📄 Indexed documents: {len(POLICY_DOCUMENTS)}")
print(f"   🧠 Memory entries: {len(memory_manager.memories)}")
print(f"   🤖 Evaluation agent: Ready for testing")

# Test the integrated system
print(f"\n🧪 Testing Integrated System:")
test_question = "What's the cut-off time for same-day domestic wire transfers?"
test_response = evaluation_agent.answer_question(test_question)

print(f"   Question: {test_response['question']}")
print(f"   Answer: {test_response['answer']}")
print(f"   Memory sources: {test_response['memory_count']} entries")
print(f"   RAG sources: {test_response['rag_count']} documents")
print(f"   Tokens used: {test_response['tokens_used']}")

print("\n🎯 Ready to implement evaluation metrics!")

## 🎯 TODO Section 1: Implement Factual Accuracy Metric

**YOUR TASK**: Implement the first metric - Factual Accuracy (40% weight).

**Learning Focus**: LLM-based evaluation for correctness assessment.

In [ ]:
def evaluate_factual_accuracy(agent_answer: str, correct_answer: str, question: str) -> Dict[str, Any]:
    """Metric 1: Factual Accuracy (40% weight) - LLM-based correctness scoring"""
    
    # TODO: Create an evaluation prompt for LLM-based scoring
    # Scoring scale: 0-100 with specific ranges:
    # - 90-100: All key facts correct, comprehensive
    # - 70-89: Most facts correct, minor missing details  
    # - 50-69: Some correct facts, significant gaps
    # - 30-49: Few correct facts, mostly incorrect
    # - 0-29: Incorrect or completely missing information
    
    prompt = f"""
    # YOUR CODE HERE: Create evaluation prompt
    # Include: question, correct_answer, agent_answer
    # Request: JSON response with accuracy_score, reasoning, key_facts_missing
    """
    
    raise NotImplementedError("TODO: Create factual accuracy evaluation prompt")
    
    try:
        # TODO: Call OpenAI API for evaluation
        # YOUR CODE HERE: Use client.chat.completions.create()
        
        raise NotImplementedError("TODO: Implement OpenAI API call for evaluation")
        
        # TODO: Parse JSON response
        # YOUR CODE HERE: Extract accuracy_score, reasoning, key_facts_missing
        
        raise NotImplementedError("TODO: Parse evaluation response")
        
    except Exception as e:
        return {
            "accuracy_score": 0,
            "accuracy_reasoning": f"Evaluation error: {str(e)}",
            "key_facts_missing": []
        }

# Test the function when implemented
print("🎯 Factual Accuracy Metric:")
print("   ⚠️ Complete the TODO sections above to test this metric!")

# Example test (uncomment after implementation):
# test_result = evaluate_factual_accuracy(
#     "The cutoff is 2 PM EST", 
#     "2:00 PM EST", 
#     "What's the wire transfer cutoff?"
# )
# print(f"   Test Score: {test_result['accuracy_score']}/100")

## 📝 TODO Section 2: Implement Citation Compliance Metric

**YOUR TASK**: Implement the second metric - Citation/Source Compliance (30% weight).

**Learning Focus**: Pattern matching and compliance checking for financial regulatory requirements.

In [ ]:
def evaluate_citation_compliance(answer: str, should_have_citation: bool, sources_used: List[str]) -> Dict[str, Any]:
    """Metric 2: Citation/Source Compliance (30% weight) - Source attribution quality"""
    
    # TODO: Define citation patterns to look for
    # Examples: [Document: title], [source], "according to", "policy states"
    citation_patterns = [
        # YOUR CODE HERE: Add regex patterns for citation detection
        # r'\[.*?\]',  # [Document: title] or [source]
        # Add more patterns...
    ]
    
    raise NotImplementedError("TODO: Define citation patterns")
    
    # TODO: Search for citations in the answer
    citations_found = []
    # YOUR CODE HERE: Use re.findall() to find citation patterns
    
    raise NotImplementedError("TODO: Implement citation detection")
    
    has_citations = len(citations_found) > 0
    
    # TODO: Calculate compliance score based on requirements
    # Rules:
    # - Required + Present = 100 points
    # - Not Required + Not Present = 100 points  
    # - Required + Missing = 0 points
    # - Not Required + Present = 80 points (acceptable)
    
    # YOUR CODE HERE: Implement compliance scoring logic
    
    raise NotImplementedError("TODO: Implement compliance scoring")
    
    # TODO: Add bonus points for citing correct sources
    # Check if source IDs appear in the answer text
    source_accuracy_bonus = 0
    # YOUR CODE HERE: Calculate source accuracy bonus
    
    raise NotImplementedError("TODO: Implement source accuracy bonus")
    
    final_score = min(100, compliance_score + source_accuracy_bonus)
    
    return {
        "citation_compliance_score": final_score,
        "citations_found": citations_found,
        "citation_expected": should_have_citation,
        "citation_present": has_citations,
        "compliance_status": compliance_status,
        "source_accuracy_bonus": source_accuracy_bonus
    }

print("📝 Citation Compliance Metric:")
print("   ⚠️ Complete the TODO sections above to test this metric!")

# Example test (uncomment after implementation):
# test_result = evaluate_citation_compliance(
#     "The cutoff is 2 PM EST [wire_transfer_policy]", 
#     True, 
#     ["wire_transfer_policy"]
# )
# print(f"   Test Score: {test_result['citation_compliance_score']}/100")

## 🔍 TODO Section 3: Implement Retrieval Relevance Metric

**YOUR TASK**: Implement the third metric - Retrieval Relevance (30% weight).

**Learning Focus**: Precision/recall analysis for RAG retrieval quality assessment.

In [ ]:
def evaluate_retrieval_relevance(question: str, retrieved_nodes: List, expected_doc_ids: List[str]) -> Dict[str, Any]:
    """Metric 3: Retrieval Relevance (30% weight) - Quality of document retrieval"""
    
    if not retrieved_nodes:
        return {
            "retrieval_relevance_score": 0,
            "retrieved_doc_ids": [],
            "expected_doc_ids": expected_doc_ids,
            "precision": 0.0,
            "recall": 0.0,
            "relevance_reasoning": "No documents retrieved"
        }
    
    # TODO: Extract document IDs from retrieved nodes
    retrieved_doc_ids = []
    # YOUR CODE HERE: Get doc_id from node.metadata for each node
    
    raise NotImplementedError("TODO: Extract retrieved document IDs")
    
    # TODO: Calculate precision and recall
    if expected_doc_ids:
        expected_set = set(expected_doc_ids)
        retrieved_set = set(retrieved_doc_ids)
        
        # YOUR CODE HERE: Calculate precision and recall
        # Precision = relevant_retrieved / total_retrieved
        # Recall = relevant_retrieved / total_relevant
        
        raise NotImplementedError("TODO: Calculate precision and recall")
        
        # TODO: Calculate F1-score as overall relevance measure
        # F1 = 2 * (precision * recall) / (precision + recall)
        
        # YOUR CODE HERE: Calculate F1-score
        
        raise NotImplementedError("TODO: Calculate F1-score")
        
        # Convert to 0-100 scale
        relevance_score = f1_score * 100
        reasoning = f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1_score:.2f}"
        
    else:
        # Handle questions with no expected documents
        precision = 0.0
        recall = 1.0 if not retrieved_doc_ids else 0.0
        relevance_score = 100 if not retrieved_doc_ids else 50
        reasoning = "No expected documents for this question"
    
    return {
        "retrieval_relevance_score": relevance_score,
        "retrieved_doc_ids": retrieved_doc_ids,
        "expected_doc_ids": expected_doc_ids,
        "precision": precision,
        "recall": recall,
        "relevance_reasoning": reasoning
    }

print("🔍 Retrieval Relevance Metric:")
print("   ⚠️ Complete the TODO sections above to test this metric!")

# Example test (uncomment after implementation):
# # Mock retrieved nodes for testing
# class MockNode:
#     def __init__(self, doc_id):
#         self.metadata = {'doc_id': doc_id}
# 
# test_nodes = [MockNode('wire_transfer_policy'), MockNode('atm_network_fees')]
# test_result = evaluate_retrieval_relevance(
#     "What's the wire transfer cutoff?",
#     test_nodes,
#     ['wire_transfer_policy']
# )
# print(f"   Test Score: {test_result['retrieval_relevance_score']:.0f}/100")
# print(f"   Precision: {test_result['precision']:.2f} | Recall: {test_result['recall']:.2f}")

## 📊 TODO Section 4: Complete Evaluation Framework

**YOUR TASK**: Combine all three metrics into a complete evaluation framework.

**Learning Focus**: Weighted scoring and comprehensive agent assessment.

In [ ]:
class AgenticRAGEvaluationFramework:
    """Complete evaluation framework for agentic RAG systems"""
    
    def __init__(self):
        self.client = client
    
    def evaluate_complete_response(self, agent_response: Dict[str, Any], gold_item: Dict[str, Any]) -> Dict[str, Any]:
        """Evaluate agent response using all three metrics"""
        
        # TODO: Use the three evaluation functions you implemented above
        
        # Metric 1: Factual Accuracy (40% weight)
        # YOUR CODE HERE: Call evaluate_factual_accuracy()
        
        raise NotImplementedError("TODO: Implement factual accuracy evaluation")
        
        # Metric 2: Citation Compliance (30% weight)  
        # YOUR CODE HERE: Call evaluate_citation_compliance()
        
        raise NotImplementedError("TODO: Implement citation compliance evaluation")
        
        # Metric 3: Retrieval Relevance (30% weight)
        # YOUR CODE HERE: Call evaluate_retrieval_relevance()
        
        raise NotImplementedError("TODO: Implement retrieval relevance evaluation")
        
        # TODO: Calculate weighted composite score
        # Weights: Accuracy 40%, Citation 30%, Retrieval 30%
        
        # YOUR CODE HERE: Calculate composite score
        
        raise NotImplementedError("TODO: Calculate weighted composite score")
        
        return {
            # Question info
            "question_id": gold_item["question_id"],
            "question": gold_item["question"],
            "category": gold_item["category"],
            "difficulty": gold_item["difficulty"],
            
            # Answers
            "agent_answer": agent_response["answer"],
            "correct_answer": gold_item["correct_answer"],
            
            # Metric scores
            "factual_accuracy_score": accuracy_eval["accuracy_score"],
            "citation_compliance_score": citation_eval["citation_compliance_score"],
            "retrieval_relevance_score": retrieval_eval["retrieval_relevance_score"],
            
            # Overall performance
            "composite_score": composite_score,
            "tokens_used": agent_response["tokens_used"]
        }

print("📊 Complete Evaluation Framework:")
print("   ⚠️ Complete all TODO sections above to use this framework!")

## 🧪 TODO Section 5: Run Evaluation Suite

**YOUR TASK**: Run the complete evaluation on a subset of questions.

**Learning Focus**: End-to-end evaluation process and results analysis.

In [ ]:
# TODO: Initialize the evaluation framework and run evaluation

# evaluator = AgenticRAGEvaluationFramework()
# evaluation_results = []

# TODO: Evaluate first 5 questions for learning purposes
# evaluation_subset = GOLDEN_QA_DATASET[:5]

# print("🧪 Running Evaluation Suite...")
# print("=" * 40)

# for i, gold_item in enumerate(evaluation_subset, 1):
#     print(f"[{i}/{len(evaluation_subset)}] Evaluating: {gold_item['question_id']}")
#     
#     # TODO: Get agent response
#     # YOUR CODE HERE: Use evaluation_agent.answer_question()
#     
#     # TODO: Evaluate the response
#     # YOUR CODE HERE: Use evaluator.evaluate_complete_response()
#     
#     # TODO: Store results and show progress
#     # YOUR CODE HERE: Append to evaluation_results and print scores

print("⚠️ Complete all previous TODO sections to run the evaluation!")
print("\n📋 When completed, you'll see:")
print("   - Individual question scores across all 3 metrics")
print("   - Composite scores with weighted averages")
print("   - Performance breakdown by category and difficulty")
print("   - Token usage and efficiency metrics")

# After completing this section, you can analyze results with:
print("\n💡 Optional: Analyze Results")
print("   results_df = pd.DataFrame(evaluation_results)")
print("   print(results_df[['question_id', 'factual_accuracy_score', 'citation_compliance_score', 'retrieval_relevance_score', 'composite_score']])")
print("   print(f'Average Composite Score: {results_df[\"composite_score\"].mean():.1f}/100')")

In [ ]:
def analyze_evaluation_results(results: List[Dict]) -> Dict[str, Any]:
    """Analyze evaluation results for insights and recommendations"""
    
    if not results:
        print("⚠️ No evaluation results available. Complete previous sections first!")
        return {}
    
    results_df = pd.DataFrame(results)
    
    # TODO: Calculate performance metrics
    performance_metrics = {
        # YOUR CODE HERE: Calculate averages for each metric
        # "avg_factual_accuracy": results_df['factual_accuracy_score'].mean(),
        # "avg_citation_compliance": ...
        # "avg_retrieval_relevance": ...
        # "avg_composite_score": ...
    }
    
    raise NotImplementedError("TODO: Calculate performance metrics")
    
    # TODO: Analyze performance by category
    # YOUR CODE HERE: Group by category and calculate mean scores
    
    raise NotImplementedError("TODO: Implement category analysis")
    
    # TODO: Analyze performance by difficulty
    # YOUR CODE HERE: Group by difficulty and calculate mean scores
    
    raise NotImplementedError("TODO: Implement difficulty analysis")
    
    # TODO: Identify problem areas
    # YOUR CODE HERE: Find questions with low scores in each metric
    
    raise NotImplementedError("TODO: Implement problem area identification")
    
    return performance_metrics

def generate_improvement_recommendations(metrics: Dict) -> List[str]:
    """Generate actionable recommendations based on evaluation results"""
    
    recommendations = []
    
    # TODO: Generate recommendations based on metric performance
    # Check each metric and suggest improvements:
    
    # if metrics.get("avg_factual_accuracy", 0) < 80:
    #     recommendations.append("🎯 Improve Factual Accuracy: ...")
    
    # YOUR CODE HERE: Add more recommendation logic
    
    raise NotImplementedError("TODO: Implement recommendation generation")
    
    return recommendations

# TODO: Run analysis when evaluation results are available
print("📈 Performance Analysis Framework Ready:")
print("   ⚠️ Complete evaluation first to generate insights!")

# Uncomment when evaluation is complete:
# metrics = analyze_evaluation_results(evaluation_results)
# recommendations = generate_improvement_recommendations(metrics)
# 
# print("\n💡 Improvement Recommendations:")
# for i, rec in enumerate(recommendations, 1):
#     print(f"{i}. {rec}")

## 🎯 Summary & Next Steps

**Congratulations!** You've learned to implement the top 3 agentic RAG evaluation metrics used in production.

### 🏆 What You've Accomplished

By completing the TODO sections, you've built:

1. **🎯 Factual Accuracy Evaluator** (40% weight)
   - LLM-based correctness scoring against golden standards
   - Critical for reliable financial advice systems

2. **📝 Citation Compliance Checker** (30% weight)
   - Source attribution quality assessment
   - Essential for regulatory compliance in financial services

3. **🔍 Retrieval Relevance Analyzer** (30% weight)
   - Precision/recall metrics for RAG document retrieval
   - Validates that the right information is found for questions

4. **📊 Complete Evaluation Framework**
   - Weighted composite scoring with industry-standard weights
   - Performance analysis by category and difficulty
   - Actionable improvement recommendations

### 💡 Key Learning Insights

- **Memory + RAG Integration**: How persistent memory enhances retrieval performance
- **Production Metrics**: Industry-standard evaluation approaches for agentic AI
- **Financial Compliance**: Citation requirements for banking applications
- **Evaluation Best Practices**: Structured assessment for continuous improvement

### 🚀 Production Deployment Considerations

Your evaluation framework is ready for:

1. **Continuous Monitoring**: Deploy metrics in production for real-time assessment
2. **A/B Testing**: Compare different agent configurations objectively  
3. **Regulatory Compliance**: Meet financial services requirements for audit trails
4. **Cost Optimization**: Monitor token usage and efficiency for budget control
5. **Quality Assurance**: Ensure consistent performance across banking domains

### 📈 Next Steps for Advanced Implementation

1. **Scale Dataset**: Expand to 500+ questions across more banking domains
2. **Hybrid Retrieval**: Implement keyword + semantic search combinations
3. **Advanced Memory**: Enhance persistent knowledge base from Concept 1
4. **Real-time Evaluation**: Deploy metrics for production monitoring
5. **Multi-agent Systems**: Evaluate collaborative agent architectures

### 🔗 Integration with Concept 1

This evaluation framework validates that your finance memory agent from Concept 1 performs well when enhanced with RAG capabilities, demonstrating the value of persistent memory in agentic AI systems!

**Ready for Production**: Your evaluation system uses the same metrics employed by leading financial institutions for AI system assessment. 🏦✨